# setup

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import cloudvolume as cv

from google.cloud import bigquery
import os
from google.oauth2 import service_account

In [2]:
import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

In [3]:
# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest
import AnalysisCode as efish 

In [4]:
'''
conda install conda-forge::google-cloud-sdk

Then, launch jupyter lab 

In a code cell, run bash command <!gcloud auth login > (https://cloud.google.com/sdk/gcloud/reference/auth/login)
    a browser tab should open up

RESULT:
You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
'''
!gcloud auth login 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=5KUZM7L0uKPEABeyX6LwGKOc91957c&access_type=offline&code_challenge=rztirOS6F3djEFTBaVhhIPE1JA1M2OOmiotkyVZ8uwo&code_challenge_method=S256


You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# get cloudvolume and bigquery client

In [5]:
efish_cloudvolume = cv.CloudVolume('gs://fish-ell/roi450um_seg32fb16fb_220930', progress=True)

In [6]:
bigquery_client = bigquery.Client(project='lcht-goog-connectomics')

/Users/kperks/opt/anaconda3/envs/ell-kimimaro/lib/python3.8/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## from file

In [7]:
df_type= pd.read_csv(Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/mg-network') / 'metadata/df_type_auto_typed.csv')

# convert state to segment locations

In [12]:
# path2statesdir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/todo')
path2skeletondir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/skeletons')

In [25]:
cell_filepaths = get_cell_filepaths(path2statesdir)

In [49]:
cell_filepaths = dict()
for child in sorted(path2statesdir.iterdir()):
    if ('json' in child.name) & (child.is_file()):
        cell_filepaths[child.name.split('_')[1].split('.')[0]] = child

In [50]:
cell_filepaths

{'393433572': PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/todo/sg2_393433572.json')}

## json from neuroglancer for 16nm VAST

In [ ]:
skeleton_folder = 'MG2_presyn_partial'

with tqdm(total=len(cell_filepaths.keys())) as pbar:
        
    for k,v in cell_filepaths.items():
        pbar.update(1)
    
        with open(v) as f:# = open(path2statesdir+'/'+cellid_filename[ind])
            data = json.load(f)
    
        # find the index of the segmentation layer
        seg_layer_idx = next(i for i, layer in enumerate(data['layers']) if layer['type'] == 'segmentation')
        
        base_seg_list = data['layers'][seg_layer_idx]['segments']
        base_seg_str = ', '.join(str(x) for x in base_seg_list)
            
        
        
        print(f'working on {k}')
        df = pd.DataFrame()
    
        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')
    
        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])

        # saveing information
        timestr = time.strftime("%Y%m%d-%H%M%S")
        savename_and_path = path2skeletondir / skeleton_folder / f'{k}_skeleton_{timestr}.csv';
        df.to_csv(savename_and_path, index=False)

print("done!")


## json from eCREST for 16nm VAST

In [8]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [9]:
directory_path = Path(settings_dict['save_dir'])#/'Rachel' # specify the directory path

cell_filepaths = efish.get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [16]:
# cell_list = ['128800360', '128816291', '128879866', '128849661', '218170254',
#        '220398612', '135733302', '135670422', '137991336', '213528973',
#        '219284058', '220336851', '216994785', '220352133', '216949671',
#        '217026667', '215820918', '214627855', '214736845', '215835078',
#        '214707822', '215897023', '218109807', '215774633', '218095913',
#        '217027723', '215883144', '216952227', '302777004', '304015064',
#        '301677416', '301693439', '300565422', '301756099', '303953883',
#        '309693374', '299467022', '562815622', '563945132', '473493287',
#        '559349295', '558189118', '473461904', '558189299', '479232204',
#        '474621530', '475766527', '480330772', '475688080', '473460724',
#        '475765678', '473491444', '475812081', '475810650', '391083807',
#        '478085373', '479230342', '476894379', '474542263', '473444334',
#        '386363638', '387634327', '474573138', '474588143', '389845032',
#        '389798775', '389891998', '387585036', '387554847', '386393755',
#        '385357762', '388730139', '388667843', '394423863', '389889717',
#        '387647235', '392102325', '386455093', '386424384', '394439668',
#        '386470356', '386501395', '389858244', '300489438', '386501395',
#        '301648806', '300489438', '300582622', '299482327', '307465674',
#        '308626960'] # CELL TYPE RATIOS data: sg1 and sg2
# cell_list = ['213590737', '559381249', '472284925', '472361842', '472282894','475781099', '386426114', '386409916', '386392158', '299392199','299439953', '298228381'] # CELL TYPE RATIOS data: mg1 and mg2
#['128770984', '132205966', '301727988', '303969144', '393325331','300474334'] # CELL TYPE RATIOS data: lg and lf 
cell_list = [128770984, 132205966, 393325331, 301727988, 303969144, 300474334]

### for only segments in a given region of each cell

In [17]:
df_type[df_type['id'].isin([301632702])]['cell_type'].values[0]

'sg1'

In [17]:
# path2statesdir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/todo')
path2skeletondir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/skeletons')

In [18]:
savename_and_path = path2skeletondir  / 'subvolume'

dtype='apical dendrite'
for ind,cell_id in enumerate(cell_list):
    # f = open(path2statesdir+'/'+cellid_filename[ind])
    # data = json.load(f)
    # crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False)
    data = efish.load_ecrest_celldata(cell_filepaths[str(cell_id)]) #crest.cell_data

    base_seg_list = [bs for bs in data['base_segments'][dtype]] # if only need segments from one cell structure #
    # [bs for dtype in data['base_segments'].keys() for bs in data['base_segments'][dtype]] # if need all segments
    # sum([i for i in data.get('base_segments').values()],[]); #
    #
    base_seg_list = ', '.join(str(x) for x in base_seg_list)
    
    # if not os.path.exists(savename_and_path):
    # print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
    df = pd.DataFrame()

    QUERY = """
    SELECT
        cast(objects.id as INT64) as seg_id,
        sample_voxel.x as x,
        sample_voxel.y as y,
        sample_voxel.z as z,
    FROM
        `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
    WHERE objects.id in {}
    """.format('('+base_seg_list+')')

    df = bigquery_client.query(QUERY).to_dataframe()
    df = df.drop_duplicates(subset=['seg_id'])
    # df = df[df['y']<15000]
    ctype = df_type[df_type['id'].isin([cell_id])]['cell_type'].values[0]
    df.to_csv(savename_and_path / f'{str(cell_id)}_{ctype}_ad.csv', index=False)

print("done!")

done!


### for all segments

In [17]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [19]:

# directory_path = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/pf_post') #settings_dict['save_dir'])# specify the directory path
directory_path = Path(settings_dict['save_dir'])

cell_filepaths = efish.get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [25]:
cell_list = ['393325331']#['301787806']#['386426114', '213605530', '126649726', '299404889'] # mg2 cells with partial presynaptic annotation

In [16]:
# base_seg_str

In [26]:
skeleton_folder = 'LF_presyn_labeled'
for k in cell_list:
    v = cell_filepaths[k]
# for k,v in cell_filepaths.items():
    # f = open(path2statesdir+'/'+cellid_filename[ind])
    # data = json.load(f)
    # crest = ecrest(settings_dict,filepath= v, launch_viewer=False)
    data = efish.load_ecrest_celldata(v)#crest.cell_data
    ctype = df_type[df_type['id'].isin([int(k)])]['cell_type'].values[0]#crest.get_ctype('manual')

    # find the index of the segmentation layer

    base_seg_list = [bs for dtype in data['base_segments'].keys() for bs in data['base_segments'][dtype]]  #sum([i for i in data.get('base_segments').values()],[]);
    base_seg_str = ', '.join(str(x) for x in base_seg_list)

    # saveing information
    savename_and_path = path2skeletondir  / skeleton_folder / f"{ctype}_{k}_skeleton.csv";

    if not os.path.exists(savename_and_path):
        # print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
        df = pd.DataFrame()

        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')

        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])
        df.to_csv(savename_and_path, index=False)
print("done!")


done!


### for 4nm resolution 

In [94]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [95]:
directory_path = Path(settings_dict['save_dir'])#/'Rachel' # specify the directory path

cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [96]:
cellid = ['368151283']

for ind,c in enumerate(cellid):
    f = open(cell_filepaths[c])
    data = json.load(f)

    # find the index of the segmentation layer

    base_seg_list = sum([i for i in data.get('base_segments').values()],[]);
    base_seg_str = ', '.join(str(x) for x in base_seg_list)

    # saveing information
    savename_and_path = path2skeletondir / f"pf_{c}_skeleton_4nm.csv";

    if not os.path.exists(savename_and_path):
        print("working on:" +c+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
        df = pd.DataFrame()

        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')

        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])
        df['x']=df['x'] *16/4
        df['y']=df['y'] *16/4
        df.to_csv(savename_and_path, index=False)
print("done!")

working on:368151283...1/1cells
done!


# other

## get segment list from neuroglancer state

In [28]:
fpath = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/MolecularLayerModel_Michal/segments/')
fname = 'segments_axon.json'
with open(fpath/fname) as f:# = open(path2statesdir+'/'+cellid_filename[ind])
    data = json.load(f)
    
    # find the index of the segmentation layer
    seg_layer_idx = next(i for i, layer in enumerate(data['layers']) if layer['type'] == 'segmentation')
    
    base_seg_list = data['layers'][seg_layer_idx]['segments']

In [29]:
base_seg_list = [item for item in base_seg_list if '!' not in item]

In [30]:
json_string = json.dumps(base_seg_list, indent=4)
listname = fname.split('.')[0] + '_list.json'
# Write the JSON string to a file
with open(fpath/listname, "w") as f:
    f.write(json_string)

## query

In [13]:
# saveing information
# savename_and_path = path2skeletondir +'/' + "skeleton_"+cellid[ind]+".csv";


# print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
df = pd.DataFrame()

QUERY = """
SELECT
    cast(objects.id as INT64) as seg_id,
    sample_voxel.x as x,
    sample_voxel.y as y,
    sample_voxel.z as z,
FROM
    `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
WHERE objects.id in {}
""".format('('+base_seg_str+')')

df = bigquery_client.query(QUERY).to_dataframe()
df = df.drop_duplicates(subset=['seg_id'])
# df.to_csv(savename_and_path, index=False)